# Pytorch Transformer

1. __Pytorch__ : Facebook AI Research에서 개발한 유연하고 사용하기 쉬운 API를 가진 딥러닝 프레임워크.  

2. __Transformer model__ : NLP에서 뛰어난 성능을 보이는 모델. Self-Attention 메커니즘을 활용해 텍스트 문맥을 파악, 병렬 처리에 강한 구조를 가짐. BERT, GPT, TS 같은 모델들이 모두 Transformer 기반이다.  

## PyTorch로 Transformer 모델 구현  

### 1. Pytorch 설치  

```
pip install torch torchvision torchaudio
```

### 2. Transformer 모델 불러오기  

- PyTorch에는 사전에 구현된 Transformer 모델이 있다. 
- d_model, nhead, num_encoder_layers, num_decoder_layers 등 파라미터는 모델의 크기, 복잡성을 조절.  

In [1]:
import torch
import torch.nn as nn
from torch.nn import Transformer

```python
model = Transformer(d_model=512, nhead=8, num_encoder_layers=6, num_decoder_layers=6)
```

### 3. 모델 학습 준비, 학습 시키기

- 학습 데이터를 준비, 옵티마이저와 손실 함수를 설정.

```python
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
```

- 모델 학습시키기

```python
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(src, tgt)
    loss = criterion(output, tgt_labels)
    loss.backward()
    optimizer.step()
```

## 사전 학습된 모델 활용하기

- Pytorch Hub를 이용하면, 사전 학습된 다양한 모델들을 활용 가능하다. 예를 들어, torch.hub.load()를 사용해 GPT-2 같은 사전 학습 모델을 불러올 수 있다.  

```python
%pip install sentencepiece sacremoses
```

- 이미 방대한 데이터셋으로 학습된 상태. 추가적 학습 없이도 다양한 작업에 활용 가능

In [2]:
model = torch.hub.load('huggingface/pytorch-transformers', 'modelForCausalLM', 'gpt2')
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'gpt2')

Using cache found in C:\Users\By the book/.cache\torch\hub\huggingface_pytorch-transformers_main
Using cache found in C:\Users\By the book/.cache\torch\hub\huggingface_pytorch-transformers_main


In [3]:
input_text = "Hello, my name is seok."

In [4]:
input_ids = tokenizer.encode(input_text, return_tensors='pt')

In [5]:
output = model.generate(input_ids, max_length = 50, num_return_sequences = 1)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [6]:
tokenizer.decode(output[0], skip_special_tokens = True)

"Hello, my name is seok. I'm a student at the University of Tokyo. I'm a student at the University of Tokyo. I'm a student at the University of Tokyo. I'm a student at the University of Tokyo. I'm"

## 문제점  

### 대형 모델 학습의 어려움  

1. __데이터 및 컴퓨팅 자원의 한계__  
- Transformer 모델은 특히나 방대한 데이터를 필요로 하고, 학습에 많은 시간이 걸림. 일반적으로 수십 기가바이트(GPU 메모리) 이상이 필요. 몇 주간의 학습 시간도 가능.  

2. __모델 크기와 메모리 사용량__  
- 모델이 커질수록 메모리 사용량도 기하급수적으로 늠. 개인이 보유한 일반적인 컴퓨터나 단일 GPU로는 이 대형 모델들을 학습시키기 매우 어려움.  

### 복잡한 모델은 직접 제작이 어려움  

1. __구현의 어려움__  
- Transformer 같은 모델은 구조가 복잡. 직접 구현에는 많은 지식, 경험이 필요. Self_attention, Position_wise Feed_Forward Networks, Multi-Head Attention 등의 여러 개념을 잘 이해해야 함.  

2. __하이퍼파라미터 튜닝__  
- 학습률, 모델 크기, 레이어 수 등 다양한 하이퍼파라미터를 적절히 조절해야함. 최적의 파라미터를 찿기 위해서는 많은 실험과 시간이 필요함.  

### 사전 학습된 모델 활용의 한계  

1. __맞춤화의 어려움__  
- 사전 학습된 모델은 특정 데이터나 작업에 대해 학습된 상태. 그래서 다른 작업에 맞추려면 추가적인 미세 조정(Fine-Tuning)이 필요함.  

2. __비용 문제__  
- 미세 조정이나 추가 학습을 하려면 대형 클라우드 서비스나 고성능 장비가 필요할 수 있으며, 이는 상당한 비용을 요구할 수 있음.  

## 극복 방법  

1. __클라우드 서비스 활용__  
- Google Colab, AWS와 같은 클라우드 기반의 GPU 서비스를 활용하면, 개인 컴퓨터의 한계 극복 가능. 무료 혹은 저렴한 비용으로 GPU 자원 사용 가능.  

2. __사전 학습된 모델을 적절히 활용__  
- Hugging Face의 Transformers 라이브러리, PyTorch Hub에서 제공하는 사전 학습된 모델을 활용 시, 복잡한 모델 처음부터 구현할 필요 X. 필요따라 일부 파라미터 Fine-Tuning으로 자신만의 모델을 만들 수 있음.  

3. __경량화 모델 사용__  
- DistilBERT, TinyBERT 같은 경량화된 모델은 대형 모델의 성능을 유지하면서도 자원 소모를 줄일 수 있는 좋은 대안이 될 수 있음.  